# 1. Import Library

In [1]:
import os
import pandas as pd
import difflib
from collections import defaultdict
import csv
import matplotlib.pyplot as plt


# 2. Load Dataset

In [2]:
PROVINCE_NAMES = [
    'Hà Giang', 'Cao Bằng', 'Lạng Sơn', 'Bắc Giang', 'Phú Thọ', 'Thái Nguyên', 'Bắc Kạn', 'Tuyên Quang', 
    'Lào Cai', 'Yên Bái', 'Lai Châu', 'Sơn La', 'Điện Biên', 'Hòa Bình', 'Hà Nội', 'Hải Phòng', 'Hải Dương', 
    'Hưng Yên', 'Vĩnh Phúc', 'Bắc Ninh', 'Thái Bình', 'Nam Định', 'Hà Nam', 'Ninh Bình', 'Quảng Ninh', 
    'Thanh Hóa', 'Nghệ An', 'Hà Tĩnh', 'Quảng Bình', 'Quảng Trị', 'Thừa Thiên - Huế', 'Đà Nẵng', 'Quảng Nam', 
    'Quảng Ngãi', 'Bình Định', 'Phú Yên', 'Khánh Hòa', 'Ninh Thuận', 'Bình Thuận', 'Kon Tum', 'Gia Lai', 
    'Đắk Lắk', 'Đắk Nông', 'Lâm Đồng', 'TP. Hồ Chí Minh', 'Đồng Nai', 'Bà Rịa - Vũng Tàu', 'Bình Dương', 
    'Bình Phước', 'Tây Ninh', 'TP. Cần Thơ', 'Long An', 'Tiền Giang', 'Bến Tre', 'Trà Vinh', 'Vĩnh Long', 
    'An Giang', 'Đồng Tháp', 'Kiên Giang', 'Hậu Giang', 'Sóc Trăng', 'Bạc Liêu', 'Cà Mau'
]

In [3]:
YEARS = [2018, 2019, 2020, 2021, 2022, 2023]

In [4]:
path_university_score = r"..\DATA\CLEAN DATA\Điểm thi THPTQG [2018-2023]\university_scores_clean.csv"
path_education_investment = r"..\DATA\CLEAN DATA\Đầu tư giáo dục [2018-2023]\clean_education_data.csv"
path_population_density = r"..\DATA\CLEAN DATA\Dân số và mật độ dân số [2018-2022]\clean_population.csv"
path_income = r"..\DATA\CLEAN DATA\Chỉ số thu nhập [2018-2020]\Chỉ số thu nhập [2018-2020].csv"
path_health_index = r"..\DATA\CLEAN DATA\Chỉ số sức khỏe [2018-2020]\Chỉ số sức khỏe [2018-2020].csv"
path_human_development_index = r"..\DATA\CLEAN DATA\Chỉ số phát triển con người [2018-2020]\Chỉ số phát triển con người [2018-2020].csv"
path_education_index = r"..\DATA\CLEAN DATA\Chỉ số giáo dục [2018-2020]\Chỉ số giáo dục [2018-2020].csv"
path_economic_regions = r"..\DATA\CLEAN DATA\6 vùng kinh tế\regions_data.csv"

# List các đường dẫn
paths = [
    path_university_score,
    path_education_investment,
    path_population_density,
    path_income,
    path_health_index,
    path_human_development_index,
    path_education_index,
    path_economic_regions
]

In [5]:
for path in paths:
    print("File:", path)
    try:
        df = pd.read_csv(path)
        print(df.info())  # In ra thông tin về cấu trúc của DataFrame
        print("Số lượng giá trị null trong từng cột:")
        print(df.isnull().sum())  # In ra số lượng giá trị null trong mỗi cột
        print("\n")
    except FileNotFoundError:
        print("File not found.")

File: DATA\CLEAN DATA\Điểm thi THPTQG [2018-2023]\university_scores_clean.csv
File not found.
File: DATA\CLEAN DATA\Đầu tư giáo dục [2018-2023]\clean_education_data.csv
File not found.
File: DATA\CLEAN DATA\Dân số và mật độ dân số [2018-2022]\clean_population.csv
File not found.
File: DATA\CLEAN DATA\Chỉ số thu nhập [2018-2020]\Chỉ số thu nhập [2018-2020].csv
File not found.
File: DATA\CLEAN DATA\Chỉ số sức khỏe [2018-2020]\Chỉ số sức khỏe [2018-2020].csv
File not found.
File: DATA\CLEAN DATA\Chỉ số phát triển con người [2018-2020]\Chỉ số phát triển con người [2018-2020].csv
File not found.
File: DATA\CLEAN DATA\Chỉ số giáo dục [2018-2020]\Chỉ số giáo dục [2018-2020].csv
File not found.
File: DATA\CLEAN DATA\6 vùng kinh tế\regions_data.csv
File not found.


# 3.Joining Tables to Create a Unified Dataset for Multivariate Relationship Analysis

In [6]:
data = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0)))

## 3.1 Process University Scores

In [7]:
df = pd.read_csv(path_university_score)

# Pivot DataFrame để có các tỉnh thành làm cột
df_pivot = df.pivot(index='Năm', columns='Địa phương', values='Giá trị')

for province in PROVINCE_NAMES:
    for year in YEARS:
        data[province][year]["university_score"] = df_pivot[province][year]

print(df_pivot['Đà Nẵng'][2023])
print(data['Đà Nẵng'][2023]['university_score'])


FileNotFoundError: [Errno 2] No such file or directory: 'DATA\\CLEAN DATA\\Điểm thi THPTQG [2018-2023]\\university_scores_clean.csv'

## 3.2 Process Economic Zone

In [ ]:
df = pd.read_csv(path_economic_regions)

for province in PROVINCE_NAMES:
    for year in YEARS:
        # Lọc dữ liệu cho tỉnh và in ra cột "Vùng kinh tế - Xã hội"
        data[province][year]["economic_zone"] = df[df["Địa phương"] == province]["Vùng kinh tế - Xã hội"].iloc[0]

print(df[df["Địa phương"] == "Đà Nẵng"]['Vùng kinh tế - Xã hội'].iloc[0])
print(data["Đà Nẵng"][2018]["economic_zone"])


Vùng Bắc Trung Bộ và duyên hải miền Trung
Vùng Bắc Trung Bộ và duyên hải miền Trung


## 3.3 Process Education Investment

In [ ]:
df = pd.read_csv(path_education_investment)

# Pivot DataFrame để có các tỉnh thành làm cột
df_pivot = df.pivot(index='Năm', columns='Địa phương', values='Chi tiêu cho giáo dục')

for province in PROVINCE_NAMES:
    for year in YEARS:
        if province in df_pivot.columns and year in df_pivot.index:
            data[province][year]["education_investment"] = df_pivot[province][year]

print(df_pivot['An Giang'][2018])
print(data['An Giang'][2018]['education_investment'])

0.698327
0.698327


## 3.4 Process Population And Population Density

In [ ]:
import pandas as pd

# Đọc dữ liệu từ file CSV
df = pd.read_csv(path_population_density)

for province in PROVINCE_NAMES:
    for year in YEARS:
        # Kiểm tra xem có tồn tại hàng thỏa mãn điều kiện 'Địa phương' và 'Năm' không
        if ((df['Địa phương'] == province) & (df['Năm'] == year)).any():
            # Lấy dữ liệu từ DataFrame cho province và year tương ứng
            popu_data = df[(df['Địa phương'] == province) & (df['Năm'] == year)]
            population = popu_data['Dân số trung bình (Nghìn người)'].values[0]
            population_density = popu_data['Mật độ dân số (Người/km2)'].values[0]
            data[province][year]["population"] = population
            data[province][year]["population_density"] = population_density

print(df[(df['Địa phương'] == 'Hà Nội') & (df['Năm'] == 2018)]['Dân số trung bình (Nghìn người)'].values[0], df[(df['Địa phương'] == 'Hà Nội') & (df['Năm'] == 2018)]['Mật độ dân số (Người/km2)'].values[0])
print(data["Hà Nội"][2018]['population'], data["Hà Nội"][2018]['population_density'])


7914.5 2356.49
7914.5 2356.49


## 3.5 Process Educational Index

In [ ]:
df = pd.read_csv(path_education_index)

# Pivot DataFrame để có các tỉnh thành làm cột
df_pivot = df.pivot(index='Năm', columns='Địa phương', values='Giá trị')

for province in PROVINCE_NAMES:
    for year in range(2018, 2021):
        data[province][year]["education_index"] = df_pivot[province][year]

print(df_pivot['Đà Nẵng'][2018])
print(data['Đà Nẵng'][2018]['education_index'])


0,745
0,745


## 3.6 Process Human Development Index

In [ ]:
df = pd.read_csv(path_human_development_index)

# Pivot DataFrame để có các tỉnh thành làm cột
df_pivot = df.pivot(index='Năm', columns='Địa phương', values='Giá trị')

for province in PROVINCE_NAMES:
    for year in range(2018, 2021):
        data[province][year]["human_development_index"] = df_pivot[province][year]

print(df_pivot['Đà Nẵng'][2018])
print(data['Đà Nẵng'][2018]['human_development_index'])


0,773
0,773


## 3.7 Process Health Index

In [ ]:
df = pd.read_csv(path_health_index)

# Pivot DataFrame để có các tỉnh thành làm cột
df_pivot = df.pivot(index='Năm', columns='Địa phương', values='Giá trị')

for province in PROVINCE_NAMES:
    for year in range(2018, 2021):
        data[province][year]["health_index"] = df_pivot[province][year]

print(df_pivot['Đà Nẵng'][2018])
print(data['Đà Nẵng'][2018]['health_index'])


0,862
0,862


## 3.8 Process Income Index

In [ ]:
df = pd.read_csv(path_income)

# Pivot DataFrame để có các tỉnh thành làm cột
df_pivot = df.pivot(index='Năm', columns='Địa phương', values='Giá trị')

for province in PROVINCE_NAMES:
    for year in range(2018, 2021):
        data[province][year]["income_index"] = df_pivot[province][year]

print(df_pivot['Đà Nẵng'][2018])
print(data['Đà Nẵng'][2018]['income_index'])


0,719
0,719


In [ ]:
print(data["Đà Nẵng"][2018]['education_index'])

0,745


In [ ]:
# Đường dẫn đến file CSV
csv_file = 'Multivariate-data.csv'

# Ghi dữ liệu vào file CSV
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Viết header
    writer.writerow(['Địa phương', 'Năm', 'Vùng kinh tế', 'Tiền đầu tư giáo dục', 
                     'Dân số trung bình (Nghìn người)', 'Mật độ dân số (Người/km2)', 'Chỉ số Giáo dục',
                     'Chỉ số phát triển con người', 'Chỉ số sức khỏe', 'Chỉ số thu nhập', 
                     'Điểm thi đại học trung bình của địa phương'])

    for province in PROVINCE_NAMES:
        for year in YEARS:
            writer.writerow(
                            [province,
                            year, 
                            data[province][year]['economic_zone'],
                            data[province][year]['education_investment'],
                            data[province][year]['population'],
                            data[province][year]['population_density'],
                            data[province][year]['education_index'],
                            data[province][year]['human_development_index'],
                            data[province][year]['health_index'],
                            data[province][year]['income_index'],
                            data[province][year]['university_score']]
                            )

In [ ]:
df = pd.read_csv(r"Multivariate-data.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378 entries, 0 to 377
Data columns (total 11 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Địa phương                                  378 non-null    object 
 1   Năm                                         378 non-null    int64  
 2   Vùng kinh tế                                378 non-null    object 
 3   Tiền đầu tư giáo dục                        378 non-null    float64
 4   Dân số trung bình (Nghìn người)             378 non-null    float64
 5   Mật độ dân số (Người/km2)                   378 non-null    float64
 6   Chỉ số Giáo dục                             378 non-null    object 
 7   Chỉ số phát triển con người                 378 non-null    object 
 8   Chỉ số sức khỏe                             378 non-null    object 
 9   Chỉ số thu nhập                             378 non-null    object 
 10  Điểm thi đại h

In [ ]:
df.isnull().sum()


Địa phương                                    0
Năm                                           0
Vùng kinh tế                                  0
Tiền đầu tư giáo dục                          0
Dân số trung bình (Nghìn người)               0
Mật độ dân số (Người/km2)                     0
Chỉ số Giáo dục                               0
Chỉ số phát triển con người                   0
Chỉ số sức khỏe                               0
Chỉ số thu nhập                               0
Điểm thi đại học trung bình của địa phương    0
dtype: int64

In [ ]:
df.describe()


,Năm,Tiền đầu tư giáo dục,Dân số trung bình (Nghìn người),Mật độ dân số (Người/km2),Điểm thi đại học trung bình của địa phương
count,378.000000,378.000000,378.000000,378.000000,378.000000
mean,2020.500000,0.725851,1289.503810,432.928069,6.026370
std,1.710089,0.852650,1439.552541,642.760858,0.967508
min,2018.000000,0.000000,0.000000,0.000000,3.690686
25%,2019.000000,0.252687,630.625000,94.135000,5.351556
50%,2020.500000,0.628391,1160.460000,217.000000,5.935970
75%,2022.000000,0.904333,1504.852500,538.750000,6.716655
max,2023.000000,10.308810,9389.720000,4481.000000,8.283732


## 4. Finding Multivariate Relationships